In [ ]:
from src.sampling.images import plot_frame_count_distributions

samples_root_dir = "data/samples"

plot_frame_count_distributions(samples_root_dir)

In [ ]:
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation
from src.sota.kfold import SOTAFoldCrossValidation
from src.common.plot import box_plot_accuracies

box_plot_accuracies(
    HpeDnnFoldCrossValidation.evaluation_instance("arch1-kf"),
    HpeDnnFoldCrossValidation.evaluation_instance("arch2-kf"), 
    HpeDnnFoldCrossValidation.evaluation_instance("arch3-kf"), 
    HpeDnnFoldCrossValidation.evaluation_instance("arch4-kf"), 
    HpeDnnFoldCrossValidation.evaluation_instance("arch5-kf"),
    SOTAFoldCrossValidation.evaluation_instance("yolo11n-kf", "yolov11n-cls"))

In [ ]:
from src.hpe.mp.landmarks import get_feature_labels as get_mp_feature_labels

mp_features = get_mp_feature_labels()
print(mp_features)

In [ ]:
from numpy import array, concatenate
from typing import List

from src.hpe.common.labels import MyLandmark

def get_name(landmark: MyLandmark):
    return landmark.name

def to_features(name: str) -> List[str]:
    return [name+"_x", name+"_y", name+"_z", name+"_visibility"]

def get_feature_labels():
    names = list(map(get_name, MyLandmark))
    features = concatenate(list(map(to_features, names)))
    return array(features)

features = get_feature_labels()
print(features)

In [ ]:
print(mp_features == features)

In [ ]:
from typing import Any, List, Callable
from ultralytics import YOLO
from ultralytics.engine.results import Results
from numpy import ndarray

from src.labels import get_label_value_from_path
from src.hpe.yolo.model import build_pose_model
from src.hpe.yolo.evaluate import predict_landmarks

def __to_np_array(results: Results) -> ndarray:
    pass

def to_feature_vector(image_path: str, model: YOLO) -> ndarray:
    _, results, _ = predict_landmarks(image_path, model)

    pass

def evaluate_images(image_paths: List[str]) -> List[List[Any]]:
    model = build_pose_model()
    
    def image_2_features(image_path: str) -> List[Any]:
        label = get_label_value_from_path(image_path)
        features = to_feature_vector(image_path, model)
        return [*features, label, image_path]
    
    return list(map(image_2_features, image_paths))

In [ ]:
from ultralytics import YOLO

from src.hpe.yolo.evaluate import predict_landmarks

_, results, _ = predict_landmarks("data/img/techniques/train/BACK_FLAG/(Climbing Analysis) Inside Flag - The Move You Never Do But Maybe Should__22__31.png", YOLO("yolo11x-pose.pt"))

In [ ]:
results.keypoints.data[0][0]

In [ ]:
from pandas import DataFrame, concat
from os.path import join, isdir, exists
from os import listdir, makedirs, mkdir
from numpy import zeros
from glob import glob
from typing import Any, List, Callable

from src.common.helpers import imread
from src.labels import get_label_value_from_path
from src.common.helpers import read_dataframe
from src.hpe.mp.model import build_holistic_model
from src.hpe.mp.evaluate import to_feature_vector
from src.hpe.mp.landmarks import get_feature_labels

def generate_hpe_feature_df(data_path,
        evaluate_func: Callable[[List[str]], List[List[Any]]],
        img_dataset_name = "techniques",
        df_dataset_name = "techniques"):

    feature_names = get_feature_labels()
    column_names = [*feature_names, "technique", "image_path"]
    
    img_path = join(data_path, "img", img_dataset_name)
    df_path = join(data_path, "df", df_dataset_name)
    if (not exists(df_path)):
        makedirs(df_path)

    for data_split in listdir(img_path):
        data_split_path = join(img_path, data_split)
        if (isdir(data_split_path)):
            image_paths = glob(data_split_path + "/**/*.*", recursive=True)
            matrix = evaluate_func(image_paths)        
            df = DataFrame(data=matrix, columns=column_names)
            df.to_pickle(join(df_path, f"{data_split}.pkl"))

In [ ]:
generate_hpe_feature_df(data_path="data")

In [ ]:
from src.common.helpers import read_dataframe

df = read_dataframe("data/df/techniques/test.pkl")

# Sota K-Fold cross validation

In [ ]:
%matplotlib inline

from src.sota.model import SOTAConstructorArgs, SOTAMultiRunTrainArgs, SOTATrainArgs
from src.sota.kfold import SOTAFoldCrossValidation

In [ ]:
cross_validation = SOTAFoldCrossValidation(
    model_args = SOTAConstructorArgs(name="yolo11n-kf", model_arch="yolo11n-cls"),
    train_run_args = SOTAMultiRunTrainArgs(
        runs=2,
        train_args=SOTATrainArgs(epochs=10, balanced=True)
    )
)

cross_validation.test_folds()

In [ ]:
# yolo11m unbalanced
cross_validation = SOTAFoldCrossValidation(
    model_args = SOTAConstructorArgs(name="yolo11m-unbalanced-kf", model_arch="yolo11m-cls"),
    train_run_args = SOTAMultiRunTrainArgs(
        runs=1,
        train_args=SOTATrainArgs(epochs=10, balanced=False)
    )
)

cross_validation.train_folds()

In [ ]:
# yolo11n unbalanced
cross_validation = SOTAFoldCrossValidation(
    model_args = SOTAConstructorArgs(name="yolo11n-unbalanced-kf", model_arch="yolo11n-cls"),
    train_run_args = SOTAMultiRunTrainArgs(
        runs=1,
        train_args=SOTATrainArgs(epochs=10, balanced=False)
    )
)

cross_validation.train_folds()

# HPE DNN K-Fold cross validation

In [ ]:
%matplotlib inline

from src.hpe_dnn.architecture import DnnArch
from src.hpe_dnn.model import HpeDnnModelInitializeArgs, HpeDnnMultiRunTrainArgs, HpeDnnTrainArgs, HpeDnnConstructorArgs
from src.hpe_dnn.kfold import HpeDnnFoldCrossValidation

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch1-kf", model_arch=DnnArch.ARCH1),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=2,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch1-dr0.3-kf", model_arch=DnnArch.ARCH1),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=2,
        model_initialize_args=HpeDnnModelInitializeArgs(dropout_rate=0.3),
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
# arch1 unbalanced
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch1-unbalanced-kf", model_arch=DnnArch.ARCH1),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=5,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=False, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
# arch1 unaugmented
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch1-unaugmented-kf", model_arch=DnnArch.ARCH1),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=5,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=False, augment=False)
    ))

cross_validation.train_folds()

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch2-kf", model_arch=DnnArch.ARCH2),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=1,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch3-kf", model_arch=DnnArch.ARCH3),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=1,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch4-kf", model_arch=DnnArch.ARCH4),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=1,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

In [ ]:
cross_validation = HpeDnnFoldCrossValidation(
    model_args=HpeDnnConstructorArgs(name="arch5-kf", model_arch=DnnArch.ARCH5),
    train_run_args=HpeDnnMultiRunTrainArgs(
        runs=2,
        train_args=HpeDnnTrainArgs(epochs=10, balanced=True, augment=True)
    ))

cross_validation.test_folds()

## Sota model training 

In [ ]:
from src.common.model import ModelInitializeArgs, TestArgs
from src.sota.model import SOTA, SOTAConstructorArgs

sota = SOTA(args=SOTAConstructorArgs("yolo11m-cls"))
sota.initialize_model(args=ModelInitializeArgs())

#sota.execute_train_runs(model="yolo11m-cls", runs=3, epochs=10, balanced=False)
sota.test_model(args=TestArgs(write_to_wandb=False))

#sota.train_model(optimizer="AdamW", lr0=0.0005)

#metrics = model.val(data="data/img/techniques/val")

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-cls")
sota.execute_train_runs(model="yolo11n-cls", runs=5, epochs=10, balanced=False)

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-cls-balanced")
sota.execute_train_runs(model="yolo11m-cls", runs=5, epochs=10, balanced=True)

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-cls-full-balanced")
sota.execute_train_runs(model="yolo11n-cls", runs=2, epochs=5, balanced=True)

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11m-cls")
sota.train_model()

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11n-balance-50-155", dataset_name="techniques_balanced")
sota.initialize_model("yolo11n-cls")
#sota.train_model()
sota.test_model()

# Sota model testing

In [ ]:
from src.sota.model import SOTA

sota = SOTA("data", "yolo11m-cls")
metrics = sota.test_model(write_to_wandb=False)

print(metrics)

In [ ]:
metrics.top1

## HPE DNN model training

In [ ]:
from src.hpe_dnn.architecture import DnnArch
from src.common.model import TestArgs
from src.hpe_dnn.model import HpeDnn, HpeDnnConstructorArgs, HpeDnnModelInitializeArgs, HpeDnnTrainArgs

In [ ]:
#Unity model for testing
hpednn = HpeDnn(args=HpeDnnConstructorArgs(name="unity", model_arch=DnnArch.ARCH1, dataset_name="unity"))
hpednn.initialize_model(args=HpeDnnModelInitializeArgs())
hpednn.train_model(args=HpeDnnTrainArgs(epochs=10))
hpednn.test_model(args=TestArgs())

In [ ]:
hpednn = HpeDnn(args=HpeDnnConstructorArgs(name="arch1-kf-fold1", model_arch=DnnArch.ARCH1))
hpednn.initialize_model(args=HpeDnnModelInitializeArgs())

hpednn.model.summary()

In [ ]:
hpednn = HpeDnn(args=HpeDnnConstructorArgs(name="arch1", model_arch=DnnArch.ARCH1))
hpednn.initialize_model(args=HpeDnnModelInitializeArgs())

hpednn.model.summary()
#hpednn.train_model(args=HpeDnnTrainArgs(epochs=10))

In [ ]:
hpednn = HpeDnn("data", "arch1-balanced")
hpednn.execute_train_runs(runs=5, epochs=10, augment=True, balanced=True)

In [ ]:
%load_ext tensorboard

from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model()

In [ ]:
from os.path import join
run_dir = join("data", "sota", "yolo11n-cls-full-balanced")
run_dir

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_balanced_augmented", "techniques_balanced")
hpednn.initialize_model()
hpednn.train_model(augment=True)

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch2_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH2)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch3_balanced", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH3)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn

hpednn = HpeDnn("data", "arch1_full_balanced")
hpednn.execute_train_runs(runs=2, epochs=10, augment=True, balanced=True)

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_not_norm", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, normalize=False)
hpednn.train_model()

In [ ]:
from src.hpe_dnn.model import HpeDnn, DnnArch

hpednn = HpeDnn("data", "arch1_balanced_dr_0.3", "techniques_balanced")
hpednn.initialize_model(DnnArch.ARCH1, dropout_rate=0.3)
hpednn.train_model()

In [ ]:
%tensorboard --logdir data/runs/hpe_dnn/arch1_balanced/train1/logs/train

In [ ]:
from src.hpe_dnn.model import read_data

df_path = "data/df/techniques/train.pkl"
train = read_data(df_path)